# Summary Statistics Generation

This notebook describes and implements the process of generating summary statistics and outputting them to a format that can be used by the companion notebook to generate synthetic data.  It makes use of the `behavioural_synthetic` library built by the Behavioural Insights Team.  

### Requirements:
- Python 3.11 or greater.
    - Note that in the actual runs in the SRS an earlier version of Python (3.7) was installed and the library had to be adapted accordingly.  This should not make any difference on the level of this notebook, but be aware that it has not yet been tested with the more advanced version of the library and contains some functions (e.g. categorical censorship) that are not present in the 3.11 version.  Caveat emptor.

### Setup: 
- Copy the following into your working directory:
    - The `behavioural_synthetic` directory.
    - The `requirements.txt` file.
- Create a venv for Python using requirements.txt.
- Set up the input and output directories.
- Edit the notebook to use those input and output directories, as described later.
- Run the relevant sections of the notebook.

## Define libraries, shared functions and shared user-defined variables.

This section describes library imports, useful functions and user-defined variables that are used throughout the notebook

### Libraries

In [ ]:
import json
import gc
from os import listdir
from os.path import isfile, join
from math import ceil

import pandas as pd

from behavioral_synthetic.tables.columns.general_functions import read_data
from behavioral_synthetic.tables.Table import Table
from behavioral_synthetic.tables.test_Table import TestTable

### Define dataset properties

The data was originally output in terms of a single, large .tsv file.  Here we define the names of data sets of interest in the form of a batch name (which sets a target directory for that batch) and a dictionary `datasets`.  `datasets` is of the form `{ ${DATASET_NAME}: {"ID": ${PROJECT_ID}},...}` where `${DATASET_NAME}` is the name of the dataset and `${PROJECT_ID}` identifies the dataset within the file.

In [ ]:
batch_name ="BATCH13" #defines target directory for batch

datasets = {

}

### Shared function

This locates whether a given file is present in a directory

In [ ]:
def is_file_in_directory(directory, filename):
    file_list = [f for f in listdir(directory) if isfile(join(directory, f))]
    return filename in file_list

## Extract datasets for individual projects from master file

This section of code finds the data sets of interest in the combined file of all projects and, if they have not already been output as a distinct .tsv file, does so.  It also ensures (using the `data_types` and `nulls`) that a list of ID variables are set to have null values, so that the final steps of synthetic data generation (in the companion notebook) can provide suitable customised values. 

- `source_file` is the path to the combined file
- `output_directory` is the path to the output directory where the individual .tsv files will be stored.

In [ ]:
set_list = [set for set in datasets]

source_file = ""
output_directory = ""

old_file_list = listdir(output_directory)
source_data_loaded = False

data_types = {
    "Anon_School_ID": "object",
    "Anon_Class_ID": "object",
    "Anon_Teacher_ID": "object",
    "Anon_Pupil_ID": "object"
}

nulls = {
    "Anon_School_ID": "",
    "Anon_Class_ID": "",
    "Anon_Teacher_ID": "",
    "Anon_Pupil_ID": ""
}

for data_set in set_list:
    output_file = f"{data_set}_original_data.tsv"
    dataset = datasets[data_set]["ID"]
    file_in_directory = is_file_in_directory(output_directory, output_file)
    
    if (not file_in_directory) and (not source_data_loaded):
        source_data = pd.read_csv(source_file, sep='\t', encoding='cp1252', low_memory=False, dtype=data_types, na_values=nulls)
        source_data_loaded = True
        
    if source_data_loaded and (not file_in_directory):
        print(f"Generating {output_file}.")
        project_data = source_data[source_data['Project']==dataset]
        project_data.to_csv(f"{output_directory}\\{output_file}", index=False, sep='\t')
    else:
        print (f"File {output_file} has already been generated.")

if source_data_loaded:
    del source_data
gc.collect()

new_files = [f for f in listdir(output_directory) if f not in old_file_list]
if len(new_files) > 0:
    new_file_list = ',\n'.join(new_files)
    print("\nThe following new input files have been generated:\n"+f"{new_file_list}.")
else:
    print("\nNo new input files generated.")

## Generate summary statistics from individual project data sets

This portion of the code generates summary statistics for the data from the .tsv files.

- You will need to set `original_data_file` to contain the path to the directory in which the .tsv files are stored and to have the correct name pattern of those files.
- You will also need to set `output_data_directory` to the path to the directory in which the summary statistics in json format will be stored.
- If `regenerate=True`, already existing output files will be overwritten, otherwise they will not.
- If `read_in_columns=True`, you will need to set `column_data_directory` to the path of a manually defined set of column definitions.  Otherwise a set of heuristics will be used to define the type of each column. The code in the Appendix can generate a column definition file from summary statistics output.
- Note that the automatic censoring of low-count categorical values has only been implemented in the SRS version of the synthetic data generation code. *If you use the non-SRS version, you should check that you are in compliance with any such disclosure requirements before requesting data release, and/or update the code with the relevant portion of the code from `summary_data/behavioural_synthetic_SRS/tables/columns/CategoricalVersion.py`.*

In most cases, this is the final step of the process.  However, in cases where a more legible format (including counts) is required for this output, the next two sections may be useful.

In [ ]:
set_list = [set for set in datasets]
regenerate = False  #Do not overwrite/regenerate existing results
read_in_columns = False #Perform heuristic analysis of columns
#regenerate = True   #Overwrite/regenerate existing results -- careful with this value!
#read_in_columns = True #Read in column values from an external file

output_data_directory = f""
old_file_list = listdir(output_data_directory)

if read_in_columns:
    column_data_directory = ""


for data_set in set_list:
    original_data_file = f""
    output_data_file = f"{data_set}_summary_stats.json"
    file_in_directory = is_file_in_directory(output_data_directory, output_data_file)

    if regenerate or (not file_in_directory):
        if regenerate and file_in_directory:
            print(f"Regenerating {output_data_file}.")
            old_file_list.remove(output_data_file)  # want the summary of new files to count regenerated ones
        else:
            print(f"Generating {output_data_file}.")
            
        original_data = pd.read_csv(original_data_file, sep='\t')
        original_table = Table(table=original_data, table_name = data_set)
        
        if read_in_columns:
            print(f"Using user defined columns")
            with open(f"{column_data_directory}\\{data_set}_column_types_checked.json", 'r') as file:
                column_data = json.load(file)
            original_table.analyse_with_column_list(columns_list=column_data)
        else:
            print(f"Using heuristic analysis of the columns")
            original_table.analyse(decimal_accuracy = 7)

        table_statistics = original_table.dictionary_out()
        #print(table_statistics)
        

        with open(f"{output_data_directory}\\{output_data_file}", 'w') as file:
            json.dump(table_statistics, file, indent=4)
    else:
        print(f"{output_data_file} already exists and regenerate is set to False.")

new_files = [f for f in listdir(output_data_directory) if f not in old_file_list]
if len(new_files) > 0:
    new_file_list = ',\n'.join(new_files)
    print("\nThe following new summary statistics files have been (re)generated:\n"+f"{new_file_list}.")
else:
    print("\nNo new summary statistics files (re)generated.")

## Generating formatted output files for SRS checking

The SRS required legible files in a non-json format alongside the counts used to generate data so that they could be easily checked before release.  For actual exported output, we required the suppression of counts as they're potentially revealing.  Code for both is provided below.

NOTE: While the code for generating files without counts doesn't currently do so, we also recommend removing any values with zero frequency (i.e. censored values) from the data you wish to export, in order to further reduce the risk of accidental disclosure.

In both cases you will need to set `output_data_directory` to the desired location of the output files, `summary_stats_file` to the path and filename pattern of the tsv files and output `output_data_file` to the desired filename pattern of the output. If `regenerate=True`, already existing output files will be overwritten, otherwise they will not.

Code for conversion into .json input is provided in `generate_data/utilities/`, consisting of `convert_from_srs.py` and `test_convert.py`.

## Formatted output file with counts (for checking)

In [ ]:
# Generate the formatted output files

ID_COLUMNS = [
    "Project_Row_ID",
    "Anon_School_ID",
    "Anon_Class_ID",
    "Anon_Teacher_ID",
    "Anon_Pupil_ID"
]

def format_column(column, number_of_rows):
    
    def is_iD(column):
        return (column["Name"] in ID_COLUMNS) and (column["Type"] != "empty")
    
    is_id_col= is_iD(column)
    
    if is_id_col:
        column["Type"] == "text"
    
    if column["Type"] == "text" or is_id_col: #is_ID(column):
        return f"""
# BEGIN COLUMN
    COLUMN NAME: {column["Name"]}
    COLUMN TYPE: "text"
# END COLUMN

"""
    elif column["Type"] == "empty":
        return f"""
# BEGIN COLUMN
    COLUMN NAME: {column["Name"]}
    COLUMN TYPE: {column["Type"]}
    
    COLUMN VALUE: {column["all_values"]}   COUNT: {number_of_rows}

# END COLUMN

"""
    elif column["Type"]=="numeric":
        return f"""
#BEGIN COLUMN
    COLUMN NAME: {column["Name"]}
    COLUMN TYPE: {column["Type"]}

    DECIMAL PRECISION: {column["decimal_precision"]}
    IS INTEGER: {column["is_integer"]}
    
    MEAN: {column["mean"]}  STANDARD DEVIATION: {column["standard_deviation"]}  COUNT {column["count for mean/standard deviation"]}
    
    AVERAGED MAXIMUM AND MINIMUM {column["averaged_max_and_min"]}
    MAXIMUM: {column["maximum"]}    COUNT: {column["# of values in average_max_min"]}
    MINIMUM: {column["minimum"]}    COUNT: {column["# of values in average_max_min"]}
    
    MISSING VALUES FREQUENCY: {column["missing_value_freq"]}  COUNT: {column["count for missing"]}
    
    
#END COLUMN
    
"""
    elif column["Type"] == "categorical":
        header = f"""
#BEGIN COLUMN
    COLUMN NAME: {column["Name"]}
    COLUMN TYPE: {column["Type"]}
    
    FREQUENCIES:
"""
        footer = f"""
        
    NOTE: {column["Note"]}   
    
#END COLUMN

"""
        key_list = [key for key in column if key not in ["Name", "Type", "Disclosure", "Note", "Counts" ]]
        output = [header]
        for key in key_list:
            format_text = f"""    VALUE: {key}        FREQUENCY: {column[key]}   COUNT: {column["Counts"][key]}"""
            output.append(format_text)
            
        output.append(footer)
        return '\n'.join(output)
    else:
        print ("Format type undefined")
    
#------------------------------------------------------------------------------------------------------------------------    

    

set_list = [set for set in datasets]
regenerate = False
#regenerate = True   #Overwrite/regenerate existing results -- careful with this value!


output_data_directory = f""
old_file_list = listdir(output_data_directory)

for data_set in set_list:
    summary_stats_file = f""
    output_data_file = f""
    file_in_directory = is_file_in_directory(output_data_directory, output_data_file)
    
    if regenerate or (not file_in_directory):
        if regenerate and file_in_directory:
            print(f"Regenerating {output_data_file}.")
            old_file_list.remove(output_data_file)  # want the summary of new files to count regenerated ones
        else:
            print(f"Generating {output_data_file}.")

        with open(summary_stats_file, 'r') as file:
            summary_stats = json.load(file)
        
        column_data = summary_stats["Column_details"]
        
        # rounds to the next highest hundred to prevent secondary disclosure
        summary_stats["Number_of_rows"] = ceil(summary_stats["Number_of_rows"]*0.01)*100 
       
        
        start = f"""
#BEGIN TABLE SUMMARY
    TABLE NAME: {summary_stats["Table_name"]}
    TABLE TYPE: {summary_stats["Table_type"]}
    NUMBER OF ROWS: {summary_stats["Number_of_rows"]}
#END TABLE SUMMARY    
        
        """
        output = [start]
        
        for column in column_data:
            output.append(format_column(column, summary_stats["Number_of_rows"]))
            
        final_output = "\n".join(output)
        
        with open(f"{output_data_directory}\\{output_data_file}", "w") as file:
            file.write(final_output)
    
               
    else:
        print(f"{output_data_file} already exists and regenerate is set to False.")
        
        
new_files = [f for f in listdir(output_data_directory) if f not in old_file_list]
if len(new_files) > 0:
    new_file_list = ',\n'.join(new_files)
    print("\nThe following formatted summary statistics files have been (re)generated:\n"+f"{new_file_list}.")
else:
    print("\nNo new formatted summary statistics files (re)generated.")

            
        


## Formatted output files without counts (for export)

In [ ]:
# Generate the formatted output files without counts
ID_COLUMNS = [
    "Project_Row_ID",
    "Anon_School_ID",
    "Anon_Class_ID",
    "Anon_Teacher_ID",
    "Anon_Pupil_ID"
]

def format_column(column, number_of_rows):
    
    def is_iD(column):
        return (column["Name"] in ID_COLUMNS) and (column["Type"] != "empty")
    
    is_id_col= is_iD(column)
    
    if is_id_col:
        column["Type"] == "text"
    
    if column["Type"] == "text" or is_id_col: 
        return f"""
# BEGIN COLUMN
    COLUMN NAME: {column["Name"]}
    COLUMN TYPE: "text"
# END COLUMN

"""
    elif column["Type"] == "empty":
        return f"""
# BEGIN COLUMN
    COLUMN NAME: {column["Name"]}
    COLUMN TYPE: {column["Type"]}
    
    COLUMN VALUE: {column["all_values"]}   

# END COLUMN

"""
    elif column["Type"]=="numeric":
        return f"""
#BEGIN COLUMN
    COLUMN NAME: {column["Name"]}
    COLUMN TYPE: {column["Type"]}

    DECIMAL PRECISION: {column["decimal_precision"]}
    IS INTEGER: {column["is_integer"]}
    
    MEAN: {column["mean"]}  STANDARD DEVIATION: {column["standard_deviation"]}  
    
    AVERAGED MAXIMUM AND MINIMUM {column["averaged_max_and_min"]}
    MAXIMUM: {column["maximum"]}    
    MINIMUM: {column["minimum"]}    
    
    MISSING VALUES FREQUENCY: {column["missing_value_freq"]}  
    
    
#END COLUMN
    
"""
    elif column["Type"] == "categorical":
        header = f"""
#BEGIN COLUMN
    COLUMN NAME: {column["Name"]}
    COLUMN TYPE: {column["Type"]}
    
    FREQUENCIES:
"""
        footer = f"""
        
    NOTE: {column["Note"]}   
    
#END COLUMN

"""
        key_list = [key for key in column if key not in ["Name", "Type", "Disclosure", "Note", "Counts" ]]
        output = [header]
        for key in key_list:
            format_text = f"""    VALUE: {key}        FREQUENCY: {column[key]}   """
            output.append(format_text)
            
        output.append(footer)
        return '\n'.join(output)
    else:
        print ("Format type undefined")
    
#------------------------------------------------------------------------------------------------------------------------    

    

set_list = [set for set in datasets]
regenerate = False
#regenerate = True   #Overwrite/regenerate existing results -- careful with this value!


output_data_directory = f""
old_file_list = listdir(output_data_directory)

for data_set in set_list:
    summary_stats_file = f""
    output_data_file = f"{data_set}_summary_stats_formatted.txt"
    file_in_directory = is_file_in_directory(output_data_directory, output_data_file)
    
    if regenerate or (not file_in_directory):
        if regenerate and file_in_directory:
            print(f"Regenerating {output_data_file}.")
            old_file_list.remove(output_data_file)  # want the summary of new files to count regenerated ones
        else:
            print(f"Generating {output_data_file}.")

        with open(summary_stats_file, 'r') as file:
            summary_stats = json.load(file)
        
        column_data = summary_stats["Column_details"]
        
        # rounds to the next highest hundred to prevent secondary disclosure
        summary_stats["Number_of_rows"] = ceil(summary_stats["Number_of_rows"]*0.01)*100 
       
        
        start = f"""
#BEGIN TABLE SUMMARY
    TABLE NAME: {summary_stats["Table_name"]}
    TABLE TYPE: {summary_stats["Table_type"]}
    NUMBER OF ROWS: {summary_stats["Number_of_rows"]}
#END TABLE SUMMARY    
        
        """
        output = [start]
        
        for column in column_data:
            output.append(format_column(column, summary_stats["Number_of_rows"]))
            
        final_output = "\n".join(output)
        
        with open(f"{output_data_directory}\\{output_data_file}", "w") as file:
            file.write(final_output)
    
               
    else:
        print(f"{output_data_file} already exists and regenerate is set to False.")
        
        
new_files = [f for f in listdir(output_data_directory) if f not in old_file_list]
if len(new_files) > 0:
    new_file_list = ',\n'.join(new_files)
    print("\nThe following formatted summary statistics files have been (re)generated:\n"+f"{new_file_list}.")
else:
    print("\nNo new formatted summary statistics files (re)generated.")

            
        


# Appendix

## Generate a description file for columns from the output file for later editing

If you wish to modify the classification of different columns, this will generate files from the json summary statistics output that can be edited and read in to change the type of a given column.

You will need to set `output_data_directory` to the desired location of the output files, `summary_stats_file` to the path and filename pattern of the tsv files and output `output_data_file` to the desired filename pattern of the output. If `regenerate=True`, already existing output files will be overwritten, otherwise they will not.

In [ ]:
def columns_from_output_dict(output_dict):
    
    if output_dict['Type'] == 'numeric':
        fields = ['Name','Type', 'decimal_precision', 'averaged_max_and_min']
    elif output_dict['Type'] in ['date', 'time', 'datetime']:
        fields = ['Name','Type', 'format', 'averaged_max_and_min']
    else:
        fields = ['Name', 'Type']
        
    return {field: output_dict[field] for field in fields}


set_list = [set for set in datasets]
regenerate = False  #Do not overwrite/regenerate existing results
regenerate = True   #Overwrite/regenerate existing results -- careful with this value!

output_data_directory = f""
old_file_list = listdir(output_data_directory)

for data_set in set_list:
    summary_stats_file = f""
    output_data_file = f"{data_set}_column_types.json"
    file_in_directory = is_file_in_directory(output_data_directory, output_data_file)

    if regenerate or (not file_in_directory):
        if regenerate and file_in_directory:
            print(f"Regenerating {output_data_file}.")
            old_file_list.remove(output_data_file)  # want the summary of new files to count regenerated ones
        else:
            print(f"Generating {output_data_file}.")

        with open(summary_stats_file, 'r') as file:
            summary_stats = json.load(file)
        
        column_data = [columns_from_output_dict(column) for column in summary_stats["Column_details"]]
        
    
        with open(f'{output_data_directory}//{output_data_file}', 'w') as file:
            json.dump(column_data, file, indent = 4)
            
    else:
        print(f"{output_data_file} already exists and regenerate is set to False.")
        
        
new_files = [f for f in listdir(output_data_directory) if f not in old_file_list]
if len(new_files) > 0:
    new_file_list = ',\n'.join(new_files)
    print("\nThe following new columns description files have been (re)generated:\n"+f"{new_file_list}.")
else:
    print("\nNo new summary statistics files (re)generated.")

        